# Test handling of OAuth 2.0 tokens with Fitbit Web API

## Import existing tokens from JSON file

In [27]:
import json
import base64

with open('tokens/tokens.json') as f:
    token_dict = json.load(f)

client_id: str = token_dict['client_id']
client_secret: str = token_dict['client_secret']
access_token: str = token_dict['access_token']
refresh_token: str = token_dict['refresh_token']
basic_token: str = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()
print(basic_token)

MjM5N1c4OjdhYjU0ZDJkZmVjNWMyZTYxYjczM2FhYzJlOThjODA5


## Use access token to request data from `heartrate` endpoint

In [28]:
import requests

date = "today"
headers = {
    "Authorization": f"Bearer {access_token}",
}
hr_dict = json.loads(requests.get(f"https://api.fitbit.com/1/user/-/activities/heart/date/{date}/1d.json", headers=headers).content)
print(hr_dict)

{'activities-heart': [{'dateTime': '2023-01-28', 'value': {'customHeartRateZones': [{'max': 90, 'min': 30, 'name': 'Below'}, {'max': 200, 'min': 90, 'name': 'Custom Zone'}, {'max': 220, 'min': 200, 'name': 'Above'}], 'heartRateZones': [{'caloriesOut': 1519.9497, 'max': 113, 'min': 30, 'minutes': 1440, 'name': 'Out of Range'}, {'caloriesOut': 0, 'max': 137, 'min': 113, 'minutes': 0, 'name': 'Fat Burn'}, {'caloriesOut': 0, 'max': 168, 'min': 137, 'minutes': 0, 'name': 'Cardio'}, {'caloriesOut': 0, 'max': 220, 'min': 168, 'minutes': 0, 'name': 'Peak'}]}}], 'activities-heart-intraday': {'dataset': [], 'datasetInterval': 1, 'datasetType': 'minute'}}


## Refresh token pair `(Access token, Refresh token)` using `Refresh token`

In [29]:
header = {
    "Authorization": f"Basic {basic_token}",
    "Content-Type": "application/x-www-form-urlencoded",
}

data = {
    "grant_type": "refresh_token",
    "client_id": client_id,
    "refresh_token": refresh_token,
}

response_dict = json.loads(requests.post("https://api.fitbit.com/oauth2/token", headers=header, data=data).content)
new_access_token = response_dict['access_token']
new_refresh_token = response_dict['refresh_token']
print(new_refresh_token)

token_dict.update({'refresh_token': new_refresh_token, 'access_token': new_access_token})

with open("tokens/tokens.json", "w") as f:
    json.dump(token_dict, f, ensure_ascii=False, indent=4)


9529f26442daafbd9810956d82bc93e007cc58a67032dc9a4192e57914e41c92
